<a href="https://colab.research.google.com/github/kushv16/Audio-Sentiment-Analysis/blob/master/JL_Corpus_Feature_Merging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [2]:
!pip install noisereduce

import pandas as pd
import numpy as np
import random
from glob import glob
import os
import librosa
import noisereduce as nr

import warnings
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
df = pd.read_csv('/content/drive/MyDrive/LY Project/Dataframes/Dataset/jl-corpus-df.csv')
print('The dataset has {} audio files'.format(df.shape[0]))
df.head(5)

The dataset has 2400 audio files


,Unnamed: 0,path,filename,dataset,duration,sample_rate,gender,age,emotion
0,0,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_angry_10a_1,JL-Corpus,2.75,44100,female,30,angry
1,1,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_angry_10a_2,JL-Corpus,2.60,44100,female,30,angry
2,2,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_angry_10b_1,JL-Corpus,2.45,44100,female,30,angry
3,3,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_angry_10b_2,JL-Corpus,2.60,44100,female,30,angry
4,4,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_angry_11a_1,JL-Corpus,2.00,44100,female,30,angry


In [4]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from scipy.io import wavfile
from tqdm import tqdm

path = '/content/drive/MyDrive/LY Project/Cleaned Datasets/cleaned_samples_jlcorpus/'

def clean_files(dataset): 
    df_clean = df[df['dataset'] == dataset].reset_index(drop=True)

    for i in tqdm(range(0, df_clean.shape[0])):
        updated_path = '/content/drive/MyDrive/LY Project/Datasets/JL-corpus/' + df_clean['path'][i].split('/')[-1]
        df_clean['path'][i] = updated_path
        y, sr = librosa.load(df_clean.path[i], sr=16000)
        y_trim, _ = librosa.effects.trim(y, top_db=20)
        y_noise_rem = nr.reduce_noise(y=y_trim, sr=sr, prop_decrease=0.1, stationary=True)
        name = (os.path.join(path)+df_clean.filename[i]+'_cleaned.wav')
        wavfile.write(name, sr, y_noise_rem)
        

In [ ]:
%%time
clean_files('JL-Corpus')

100%|██████████| 2400/2400 [17:02<00:00,  2.35it/s]

CPU times: user 4min 28s, sys: 13.7 s, total: 4min 41s
Wall time: 17min 2s


In [5]:
cd "/content/drive/MyDrive/LY Project/Cleaned Datasets/cleaned_samples_jlcorpus"

/content/drive/.shortcut-targets-by-id/1ty6Q2chm7khBo3iSNsdx5NVqeOpCUmjM/LY Project/Cleaned Datasets/cleaned_samples_jlcorpus


In [17]:
# saving the new cleaned files path in a variable
cleaned_files = sorted(filter(os.path.isfile, glob('*.wav')),key=os.path.getmtime)
print(len(cleaned_files))

2400


In [23]:
f0_mean, f0_median, f0_std, f0_0, f0_25, f0_75, f0_100, fname = [], [], [], [], [], [], [], []

# Function to extract f0 values
def get_f0(file):
    # load audio file:
    y, sr = librosa.load(file, sr=16000)
    
    # Extract fundamental frequency using a probabilistic approach
    f_zero, _, _ = librosa.pyin(y, sr=sr, fmin=50, fmax=1500, frame_length=1024)
    fname.append(file)
    f0_mean.append(np.nanmean(f_zero))             # f0 mean
    f0_median.append(np.nanmedian(f_zero))         # f0 median
    f0_std.append(np.nanstd(f_zero))               # f0 standar deviation
    f0_0.append(np.nanpercentile(f_zero, 0))       # f0 min
    f0_25.append(np.nanpercentile(f_zero, 25))     # f0 25%-percentile value
    f0_75.append(np.nanpercentile(f_zero, 75))     # f0 75%-percentile value
    f0_100.append(np.nanpercentile(f_zero, 100))   # f0 max

In [24]:
from tqdm import tqdm

In [25]:
%%time

count = 0
# Apply function to cleaned files
for file in tqdm(cleaned_files):
    get_f0(file)

100%|██████████| 2400/2400 [24:44<00:00,  1.62it/s]

CPU times: user 23min 2s, sys: 38.5 s, total: 23min 41s
Wall time: 24min 44s


In [26]:
df_f0 = pd.DataFrame()
df_f0['f0_mean'] = f0_mean
df_f0['f0_median'] = f0_median
df_f0['f0_std'] = f0_std
df_f0['f0_0'] = f0_0
df_f0['f0_25'] = f0_25
df_f0['f0_75'] = f0_75
df_f0['f0_100'] = f0_100
df_f0['fname'] = fname
df_f0.head()

,f0_mean,f0_median,f0_std,f0_0,f0_25,f0_75,f0_100,fname
0,219.177008,216.220905,43.332156,129.683955,193.467046,245.171366,324.900959,female1_angry_10a_1_cleaned.wav
1,233.821368,223.845919,67.100423,128.194408,185.269329,271.634489,379.736848,female1_angry_10a_2_cleaned.wav
2,201.024166,185.531823,51.878638,129.683955,163.391545,213.739184,324.900959,female1_angry_10b_1_cleaned.wav
3,273.740375,270.069989,64.761666,156.916820,231.747560,318.410406,384.149178,female1_angry_11a_1_cleaned.wav
4,223.694014,218.101547,57.499029,125.266444,181.294271,265.430348,332.495158,female1_angry_10b_2_cleaned.wav


In [27]:
df_f0.to_csv("df_fund_jlcorpus.csv")

In [28]:
df_f0.isnull().sum()

f0_mean      0
f0_median    0
f0_std       0
f0_0         0
f0_25        0
f0_75        0
f0_100       0
fname        0
dtype: int64

In [30]:
# declare variables where to store the extracted feaures
zcr_mean, zcr_var, spectral_centroid_mean, spectral_centroid_var, rms_mean, rms_var, chroma_stft_mean, chroma_stft_var = [], [], [], [], [], [], [], []
rolloff_mean, rolloff_var, spectral_bandwidth_mean, spectral_bandwidth_var, harmony_mean, harmony_var, chroma_cqt_mean, chroma_cqt_var = [], [], [], [], [], [], [], []
spectral_contrast_mean, spectral_contrast_var, spectral_flatness_mean, spectral_flatness_var, chroma_cens_mean, chroma_cens_var = [], [], [], [], [], []
fname = []
# extract features
for file in tqdm(cleaned_files):
    # load audio file:
    y, sr = librosa.load(file, sr=16000)
    fname.append(file)
    # Zero crossing rate
    zcr_mean.append(np.mean(librosa.feature.zero_crossing_rate(y)))
    zcr_var.append(np.var(librosa.feature.zero_crossing_rate(y)))
    
    # Spectral centroid
    spectral_centroid_mean.append(np.mean(librosa.feature.spectral_centroid(y)))
    spectral_centroid_var.append(np.var(librosa.feature.spectral_centroid(y)))
    
    # Spectral contrast
    spectral_contrast_mean.append(np.mean(librosa.feature.spectral_contrast(y)))
    spectral_contrast_var.append(np.var(librosa.feature.spectral_contrast(y)))
    
    # Spectral flatness
    spectral_flatness_mean.append(np.mean(librosa.feature.spectral_flatness(y)))
    spectral_flatness_var.append(np.var(librosa.feature.spectral_flatness(y)))
    
    # Rms mean and variance
    rms_mean.append(np.mean(librosa.feature.rms(y=y)))               
    rms_var.append(np.var(librosa.feature.rms(y=y)))
    
    # Chroma mean and variance
    chroma_stft_mean.append(np.mean(librosa.feature.chroma_stft(y, sr)))                   
    chroma_stft_var.append(np.var(librosa.feature.chroma_stft(y, sr)))
    
    #Chroma cqt mean and variance
    chroma_cqt_mean.append(np.mean(librosa.feature.chroma_cqt(y, sr)))                   
    chroma_cqt_var.append(np.var(librosa.feature.chroma_cqt(y, sr)))
    
    #Chroma cens mean and variance
    chroma_cens_mean.append(np.mean(librosa.feature.chroma_cens(y, sr)))                   
    chroma_cens_var.append(np.var(librosa.feature.chroma_cens(y, sr)))
    
    # Rolloff mean and variance
    rolloff_mean.append(np.mean(librosa.feature.spectral_rolloff(y,sr)))
    rolloff_var.append(np.var(librosa.feature.spectral_rolloff(y,sr)))
    
    # Spectral bandwidth mean and variance
    spectral_bandwidth_mean.append(np.mean(librosa.feature.spectral_bandwidth(y,sr)))
    spectral_bandwidth_var.append(np.var(librosa.feature.spectral_bandwidth(y,sr)))
    
    # Harmony mean and variance
    harmony_mean.append(np.mean(librosa.effects.harmonic(y)))
    harmony_var.append(np.var(librosa.effects.harmonic(y)))

100%|██████████| 2400/2400 [23:48<00:00,  1.68it/s]


In [31]:
# add new features to a new dataframe
df_features = pd.DataFrame()
df_features['zcr_mean'] = zcr_mean
df_features['zcr_var'] = zcr_var
df_features['spectral_centroid_mean'] = spectral_centroid_mean
df_features['spectral_centroid_var'] = spectral_centroid_var
df_features['spectral_contrast_mean'] = spectral_contrast_mean
df_features['spectral_contrast_var'] = spectral_contrast_var
df_features['spectral_flatness_mean'] = spectral_flatness_mean
df_features['spectral_flatness_var'] = spectral_flatness_var
df_features['rms_mean'] = rms_mean
df_features['rms_var'] = rms_var
df_features['chroma_stft_mean'] = chroma_stft_mean
df_features['chroma_stft_var'] = chroma_stft_var
df_features['chroma_cqt_mean'] = chroma_cqt_mean
df_features['chroma_cqt_var'] = chroma_cqt_var
df_features['chroma_cens_mean'] = chroma_cens_mean
df_features['chroma_cens_var'] = chroma_cens_var
df_features['spectral_bandwidth_mean'] = spectral_bandwidth_mean
df_features['spectral_bandwidth_var'] = spectral_bandwidth_var
df_features['rolloff_mean'] = rolloff_mean
df_features['rolloff_var'] = rolloff_var
df_features['harmony_mean'] = harmony_mean
df_features['harmony_var'] = harmony_var
df_features['filename'] = fname
# Check a random row
df_features.sample()

,zcr_mean,zcr_var,spectral_centroid_mean,spectral_centroid_var,spectral_contrast_mean,spectral_contrast_var,spectral_flatness_mean,spectral_flatness_var,rms_mean,rms_var,...,chroma_cqt_var,chroma_cens_mean,chroma_cens_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,harmony_mean,harmony_var,filename
687,0.147715,0.013618,2583.797741,2.947909e+06,24.14792,135.705729,0.030052,0.003137,0.00809,0.00002,...,0.091261,0.256745,0.017415,1480.191505,252613.8612,3033.365885,3.843490e+06,9.426711e-07,0.000015,female2_anxious_1b_2_cleaned.wav


In [32]:
pwd

'/content/drive/.shortcut-targets-by-id/1ty6Q2chm7khBo3iSNsdx5NVqeOpCUmjM/LY Project/Cleaned Datasets/cleaned_samples_jlcorpus'

In [37]:
df_features.to_csv("/content/drive/MyDrive/LY Project/Dataframes/Features/JL-Corpus/df_misc_jlcorpus.csv", index=False)

In [38]:
df_f0.to_csv("/content/drive/MyDrive/LY Project/Dataframes/Features/JL-Corpus/df_fund_jlcorpus.csv", index=False)

### <a id='2_b3'>MFCCs</a>

In [39]:
# function to extract mean and variance of Mel-Frequency Cepstrum Components (MFCCs)
def extract_mfcc(file):
    
    mfcc_mean, mfcc_var = [], []
    
    # load audio file:
    y, sr = librosa.load(file, sr=16000)
    
    # get 20 mean values
    mfcc_mean.append(np.mean(librosa.feature.mfcc(y=y, sr=sr, fmin=50, n_mfcc=30).T, axis=0))
    
    # get 20 variance values
    mfcc_var.append(np.var(librosa.feature.mfcc(y=y, sr=sr, fmin=50, n_mfcc=30).T, axis=0))
    
    return np.hstack((mfcc_mean, mfcc_var))[0]

# apply function above to all files and store results in a list
extracted_mfcc = []
fname = []
for file in cleaned_files:
    fname.append(file)
    extracted_mfcc.append(extract_mfcc(file))

# create column names for a dataframe
name_mfcc_mean, name_mfcc_var = [], []

for i in range(0, 30):
    name_mfcc_mean.append('mfcc'+str(i+1)+'_mean')   # mfcc1_mean, mfcc2_mean, ...
    name_mfcc_var.append('mfcc'+str(i+1)+'_var')     # mfcc1_var, mfcc2_var, ...

name_mfcc = name_mfcc_mean + name_mfcc_var           # concatenate mfcc mean and variance names

# finally put all mfcc values in a dataframe
df_mfcc = pd.DataFrame(extracted_mfcc, columns = name_mfcc)
df_mfcc['filename'] = fname
df_mfcc.sample()      # print one sample

,mfcc1_mean,mfcc2_mean,mfcc3_mean,mfcc4_mean,mfcc5_mean,mfcc6_mean,mfcc7_mean,mfcc8_mean,mfcc9_mean,mfcc10_mean,...,mfcc22_var,mfcc23_var,mfcc24_var,mfcc25_var,mfcc26_var,mfcc27_var,mfcc28_var,mfcc29_var,mfcc30_var,filename
1208,-305.625244,69.363449,-9.398084,27.964653,-2.002944,15.576597,-14.982499,7.133821,-15.108278,7.38735,...,42.445827,25.264206,51.645267,46.206169,57.605572,64.967651,39.644493,109.723907,106.396255,male1_angry_12a_1_cleaned.wav


In [41]:
df_mfcc.head()

,mfcc1_mean,mfcc2_mean,mfcc3_mean,mfcc4_mean,mfcc5_mean,mfcc6_mean,mfcc7_mean,mfcc8_mean,mfcc9_mean,mfcc10_mean,...,mfcc22_var,mfcc23_var,mfcc24_var,mfcc25_var,mfcc26_var,mfcc27_var,mfcc28_var,mfcc29_var,mfcc30_var,filename
0,-420.362488,95.686562,-6.642638,18.245741,-13.581323,-3.006383,0.091751,-8.563185,-11.144691,0.053608,...,292.119537,364.214752,290.306000,293.173981,268.178772,383.548035,507.072174,298.342468,257.717346,female1_angry_10a_1_cleaned.wav
1,-388.405731,101.115005,-10.674383,16.987240,-19.859846,-1.124406,-2.243831,-8.980619,-14.223404,0.834826,...,181.909775,184.546814,179.151260,130.375870,222.848434,209.256775,273.201355,121.195908,113.026375,female1_angry_10a_2_cleaned.wav
2,-433.245758,100.370415,-6.039964,30.019053,-16.861738,2.675807,1.483774,-7.663345,-16.894356,3.326378,...,219.164551,139.534698,113.843193,52.278015,74.257080,135.962677,191.184113,140.034317,116.458626,female1_angry_10b_1_cleaned.wav
3,-349.324677,70.172966,-36.437515,19.868406,-24.183359,-2.716273,-10.472356,-5.674997,-17.190386,-7.147378,...,217.579163,242.317856,246.153824,159.066208,113.589668,138.295837,196.050430,108.374786,187.551956,female1_angry_11a_1_cleaned.wav
4,-427.856567,102.441833,-10.381323,23.397377,-20.766310,1.868496,0.869907,-9.554515,-16.839457,2.288533,...,376.755859,126.861664,133.374680,215.111160,228.073807,208.395248,220.419281,158.635254,142.688309,female1_angry_10b_2_cleaned.wav


In [43]:
df_mfcc.to_csv("/content/drive/MyDrive/LY Project/Dataframes/Features/JL-Corpus/df_mfcc_jlcorpus.csv", index=False)

In [50]:
df_mfcc = pd.read_csv("/content/drive/MyDrive/LY Project/Dataframes/Features/JL-Corpus/df_mfcc_jlcorpus.csv")
df_fund = pd.read_csv("/content/drive/MyDrive/LY Project/Dataframes/Features/JL-Corpus/df_fund_jlcorpus.csv")
df_misc =  pd.read_csv("/content/drive/MyDrive/LY Project/Dataframes/Features/JL-Corpus/df_misc_jlcorpus.csv")

merged_df = pd.merge(df_fund, df_misc,  left_on='fname', right_on='filename', suffixes=('_fund', '_misc'))
merged_df = pd.merge(merged_df, df_misc, on='filename')


# Drop the duplicate filename columns
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]

# Save the merged dataframe to a CSV file
# merged_df.to_csv('path/to/merged_df.csv', index=False)
print(merged_df.columns)
merged_df.head()

Index(['mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean', 'mfcc4_mean', 'mfcc5_mean',
       'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean', 'mfcc9_mean', 'mfcc10_mean',
       'mfcc11_mean', 'mfcc12_mean', 'mfcc13_mean', 'mfcc14_mean',
       'mfcc15_mean', 'mfcc16_mean', 'mfcc17_mean', 'mfcc18_mean',
       'mfcc19_mean', 'mfcc20_mean', 'mfcc21_mean', 'mfcc22_mean',
       'mfcc23_mean', 'mfcc24_mean', 'mfcc25_mean', 'mfcc26_mean',
       'mfcc27_mean', 'mfcc28_mean', 'mfcc29_mean', 'mfcc30_mean', 'mfcc1_var',
       'mfcc2_var', 'mfcc3_var', 'mfcc4_var', 'mfcc5_var', 'mfcc6_var',
       'mfcc7_var', 'mfcc8_var', 'mfcc9_var', 'mfcc10_var', 'mfcc11_var',
       'mfcc12_var', 'mfcc13_var', 'mfcc14_var', 'mfcc15_var', 'mfcc16_var',
       'mfcc17_var', 'mfcc18_var', 'mfcc19_var', 'mfcc20_var', 'mfcc21_var',
       'mfcc22_var', 'mfcc23_var', 'mfcc24_var', 'mfcc25_var', 'mfcc26_var',
       'mfcc27_var', 'mfcc28_var', 'mfcc29_var', 'mfcc30_var', 'filename',
       'f0_mean', 'f0_median', 'f0_std', 'f0_

,mfcc1_mean,mfcc2_mean,mfcc3_mean,mfcc4_mean,mfcc5_mean,mfcc6_mean,mfcc7_mean,mfcc8_mean,mfcc9_mean,mfcc10_mean,...,chroma_cqt_mean,chroma_cqt_var,chroma_cens_mean,chroma_cens_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,harmony_mean,harmony_var
0,-420.36250,95.686560,-6.642638,18.245740,-13.581323,-3.006383,0.091751,-8.563185,-11.144691,0.053608,...,0.334739,0.096213,0.250667,0.020500,1393.425434,196347.497573,2523.965372,2.731156e+06,1.116470e-05,0.000129
1,-388.40573,101.115005,-10.674383,16.987240,-19.859846,-1.124407,-2.243831,-8.980619,-14.223404,0.834826,...,0.403176,0.089169,0.276131,0.007085,1362.274035,193655.067211,2514.832428,2.797436e+06,1.579629e-05,0.000078
2,-433.24576,100.370415,-6.039964,30.019053,-16.861738,2.675807,1.483774,-7.663345,-16.894356,3.326378,...,0.395417,0.095275,0.256601,0.017489,1381.380166,155367.790971,2523.437500,2.532704e+06,2.485707e-06,0.000062
3,-349.32468,70.172966,-36.437515,19.868406,-24.183360,-2.716273,-10.472356,-5.674997,-17.190386,-7.147378,...,0.408103,0.095242,0.271930,0.009387,1442.863376,128686.289602,3337.812500,2.842775e+06,-8.375696e-07,0.000046
4,-427.85657,102.441830,-10.381323,23.397377,-20.766310,1.868496,0.869907,-9.554515,-16.839457,2.288533,...,0.393002,0.091138,0.270721,0.010043,1372.388193,187456.892776,2559.818097,2.849722e+06,-3.226883e-06,0.000038


In [59]:
df_mfcc = pd.read_csv("/content/drive/MyDrive/LY Project/Dataframes/Features/JL-Corpus/df_mfcc_jlcorpus.csv")
df_fund = pd.read_csv("/content/drive/MyDrive/LY Project/Dataframes/Features/JL-Corpus/df_fund_jlcorpus.csv")
df_misc =  pd.read_csv("/content/drive/MyDrive/LY Project/Dataframes/Features/JL-Corpus/df_misc_jlcorpus.csv")

merged_df = pd.merge(df_fund, df_misc,  left_on='fname', right_on='filename', suffixes=('_fund', '_misc'))

print(merged_df.columns)
print(merged_df.shape)
merged_df.head(5)

Index(['f0_mean', 'f0_median', 'f0_std', 'f0_0', 'f0_25', 'f0_75', 'f0_100',
       'fname', 'zcr_mean', 'zcr_var', 'spectral_centroid_mean',
       'spectral_centroid_var', 'spectral_contrast_mean',
       'spectral_contrast_var', 'spectral_flatness_mean',
       'spectral_flatness_var', 'rms_mean', 'rms_var', 'chroma_stft_mean',
       'chroma_stft_var', 'chroma_cqt_mean', 'chroma_cqt_var',
       'chroma_cens_mean', 'chroma_cens_var', 'spectral_bandwidth_mean',
       'spectral_bandwidth_var', 'rolloff_mean', 'rolloff_var', 'harmony_mean',
       'harmony_var', 'filename'],
      dtype='object')
(2400, 31)


,f0_mean,f0_median,f0_std,f0_0,f0_25,f0_75,f0_100,fname,zcr_mean,zcr_var,...,chroma_cqt_var,chroma_cens_mean,chroma_cens_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,harmony_mean,harmony_var,filename
0,219.177008,216.220905,43.332156,129.683955,193.467046,245.171366,324.900959,female1_angry_10a_1_cleaned.wav,0.103509,0.011014,...,0.096213,0.250667,0.020500,1393.425434,196347.497573,2523.965372,2.731156e+06,1.116470e-05,0.000129,female1_angry_10a_1_cleaned.wav
1,233.821368,223.845919,67.100423,128.194408,185.269329,271.634489,379.736848,female1_angry_10a_2_cleaned.wav,0.117923,0.011924,...,0.089169,0.276131,0.007085,1362.274035,193655.067211,2514.832428,2.797436e+06,1.579629e-05,0.000078,female1_angry_10a_2_cleaned.wav
2,201.024166,185.531823,51.878638,129.683955,163.391545,213.739184,324.900959,female1_angry_10b_1_cleaned.wav,0.109007,0.010337,...,0.095275,0.256601,0.017489,1381.380166,155367.790971,2523.437500,2.532704e+06,2.485707e-06,0.000062,female1_angry_10b_1_cleaned.wav
3,273.740375,270.069989,64.761666,156.916820,231.747560,318.410406,384.149178,female1_angry_11a_1_cleaned.wav,0.170557,0.024981,...,0.095242,0.271930,0.009387,1442.863376,128686.289602,3337.812500,2.842775e+06,-8.375696e-07,0.000046,female1_angry_11a_1_cleaned.wav
4,223.694014,218.101547,57.499029,125.266444,181.294271,265.430348,332.495158,female1_angry_10b_2_cleaned.wav,0.116036,0.011352,...,0.091138,0.270721,0.010043,1372.388193,187456.892776,2559.818097,2.849722e+06,-3.226883e-06,0.000038,female1_angry_10b_2_cleaned.wav


In [60]:
merged_df.sample(5)

,f0_mean,f0_median,f0_std,f0_0,f0_25,f0_75,f0_100,fname,zcr_mean,zcr_var,...,chroma_cqt_var,chroma_cens_mean,chroma_cens_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,harmony_mean,harmony_var,filename
533,181.253631,183.932024,18.055180,148.109755,171.616082,191.798022,224.492410,female1_neutral_8a_2_cleaned.wav,0.206062,0.039589,...,0.098854,0.260762,0.015337,1474.995770,319667.386501,3369.140625,5.328413e+06,3.002174e-08,0.000016,female1_neutral_8a_2_cleaned.wav
1361,129.642957,131.950791,15.657869,78.912915,121.350889,141.626168,152.449702,male1_apologetic_5a_2_cleaned.wav,0.144809,0.023142,...,0.086304,0.234348,0.028414,1549.501886,185668.626483,3249.822443,4.485758e+06,-3.936007e-06,0.000274,male1_apologetic_5a_2_cleaned.wav
1901,147.386216,152.449702,22.347286,102.337389,129.683955,165.290064,186.606598,male2_anxious_5a_2_cleaned.wav,0.182407,0.042410,...,0.092645,0.261894,0.014745,1635.187633,231783.664331,3721.200980,4.837322e+06,2.574268e-06,0.000163,male2_anxious_5a_2_cleaned.wav
2074,97.514368,96.037295,15.377926,71.946679,87.056509,108.110453,131.950791,male2_concerned_3b_1_cleaned.wav,0.100242,0.020568,...,0.077416,0.248420,0.021621,1613.613452,191252.751038,3274.594907,2.977504e+06,-1.826389e-07,0.000051,male2_concerned_3b_1_cleaned.wav
1056,232.743838,227.100886,37.746841,181.294271,200.000000,256.388816,326.783091,female2_happy_4a_1_cleaned.wav,0.197003,0.022437,...,0.093457,0.240851,0.025324,1524.954501,118272.082278,3821.664663,1.903376e+06,-1.166101e-06,0.000077,female2_happy_4a_1_cleaned.wav


In [61]:
merged_df = pd.merge(merged_df, df_mfcc, on='filename')

print(merged_df.columns)
print(merged_df.shape)
merged_df.head(5)

Index(['f0_mean', 'f0_median', 'f0_std', 'f0_0', 'f0_25', 'f0_75', 'f0_100',
       'fname', 'zcr_mean', 'zcr_var', 'spectral_centroid_mean',
       'spectral_centroid_var', 'spectral_contrast_mean',
       'spectral_contrast_var', 'spectral_flatness_mean',
       'spectral_flatness_var', 'rms_mean', 'rms_var', 'chroma_stft_mean',
       'chroma_stft_var', 'chroma_cqt_mean', 'chroma_cqt_var',
       'chroma_cens_mean', 'chroma_cens_var', 'spectral_bandwidth_mean',
       'spectral_bandwidth_var', 'rolloff_mean', 'rolloff_var', 'harmony_mean',
       'harmony_var', 'filename', 'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc14_mean', 'mfcc15_mean', 'mfcc16_mean',
       'mfcc17_mean', 'mfcc18_mean', 'mfcc19_mean', 'mfcc20_mean',
       'mfcc21_mean', 'mfcc22_mean', 'mfcc23_mean', 'mfcc24_mean',
       'mfcc25_mean', 'mfcc26_m

,f0_mean,f0_median,f0_std,f0_0,f0_25,f0_75,f0_100,fname,zcr_mean,zcr_var,...,mfcc21_var,mfcc22_var,mfcc23_var,mfcc24_var,mfcc25_var,mfcc26_var,mfcc27_var,mfcc28_var,mfcc29_var,mfcc30_var
0,219.177008,216.220905,43.332156,129.683955,193.467046,245.171366,324.900959,female1_angry_10a_1_cleaned.wav,0.103509,0.011014,...,131.51295,292.11954,364.214750,290.30600,293.173980,268.17877,383.54803,507.07217,298.34247,257.717350
1,233.821368,223.845919,67.100423,128.194408,185.269329,271.634489,379.736848,female1_angry_10a_2_cleaned.wav,0.117923,0.011924,...,263.28800,181.90977,184.546810,179.15126,130.375870,222.84843,209.25677,273.20135,121.19591,113.026375
2,201.024166,185.531823,51.878638,129.683955,163.391545,213.739184,324.900959,female1_angry_10b_1_cleaned.wav,0.109007,0.010337,...,110.76114,219.16455,139.534700,113.84319,52.278015,74.25708,135.96268,191.18411,140.03432,116.458626
3,273.740375,270.069989,64.761666,156.916820,231.747560,318.410406,384.149178,female1_angry_11a_1_cleaned.wav,0.170557,0.024981,...,295.06910,217.57916,242.317860,246.15382,159.066210,113.58967,138.29584,196.05043,108.37479,187.551960
4,223.694014,218.101547,57.499029,125.266444,181.294271,265.430348,332.495158,female1_angry_10b_2_cleaned.wav,0.116036,0.011352,...,283.61505,376.75586,126.861664,133.37468,215.111160,228.07380,208.39525,220.41928,158.63525,142.688310


In [62]:
col = merged_df.pop('filename')
col1 = merged_df.pop('fname')
merged_df.insert(0, 'filename', col)
merged_df.insert(0, 'fname', col1)

merged_df.head(5)

,fname,filename,f0_mean,f0_median,f0_std,f0_0,f0_25,f0_75,f0_100,zcr_mean,...,mfcc21_var,mfcc22_var,mfcc23_var,mfcc24_var,mfcc25_var,mfcc26_var,mfcc27_var,mfcc28_var,mfcc29_var,mfcc30_var
0,female1_angry_10a_1_cleaned.wav,female1_angry_10a_1_cleaned.wav,219.177008,216.220905,43.332156,129.683955,193.467046,245.171366,324.900959,0.103509,...,131.51295,292.11954,364.214750,290.30600,293.173980,268.17877,383.54803,507.07217,298.34247,257.717350
1,female1_angry_10a_2_cleaned.wav,female1_angry_10a_2_cleaned.wav,233.821368,223.845919,67.100423,128.194408,185.269329,271.634489,379.736848,0.117923,...,263.28800,181.90977,184.546810,179.15126,130.375870,222.84843,209.25677,273.20135,121.19591,113.026375
2,female1_angry_10b_1_cleaned.wav,female1_angry_10b_1_cleaned.wav,201.024166,185.531823,51.878638,129.683955,163.391545,213.739184,324.900959,0.109007,...,110.76114,219.16455,139.534700,113.84319,52.278015,74.25708,135.96268,191.18411,140.03432,116.458626
3,female1_angry_11a_1_cleaned.wav,female1_angry_11a_1_cleaned.wav,273.740375,270.069989,64.761666,156.916820,231.747560,318.410406,384.149178,0.170557,...,295.06910,217.57916,242.317860,246.15382,159.066210,113.58967,138.29584,196.05043,108.37479,187.551960
4,female1_angry_10b_2_cleaned.wav,female1_angry_10b_2_cleaned.wav,223.694014,218.101547,57.499029,125.266444,181.294271,265.430348,332.495158,0.116036,...,283.61505,376.75586,126.861664,133.37468,215.111160,228.07380,208.39525,220.41928,158.63525,142.688310


In [63]:
merged_df.sample(10)

,fname,filename,f0_mean,f0_median,f0_std,f0_0,f0_25,f0_75,f0_100,zcr_mean,...,mfcc21_var,mfcc22_var,mfcc23_var,mfcc24_var,mfcc25_var,mfcc26_var,mfcc27_var,mfcc28_var,mfcc29_var,mfcc30_var
1386,male1_assertive_11b_1_cleaned.wav,male1_assertive_11b_1_cleaned.wav,110.468296,111.599714,19.864362,58.438862,103.526492,120.302504,156.916820,0.106676,...,48.752678,45.957570,50.412395,27.531332,22.316310,28.751013,34.611282,47.520813,31.459484,24.954850
2343,male2_sad_10b_2_cleaned.wav,male2_sad_10b_2_cleaned.wav,96.881288,93.843800,15.328736,72.782659,84.576768,105.641207,137.395365,0.073819,...,41.489360,36.182983,29.105852,18.775648,13.534279,33.352123,18.805664,27.121199,22.428574,12.294296
1287,male1_anxious_1b_2_cleaned.wav,male1_anxious_1b_2_cleaned.wav,132.243432,126.721970,41.714861,65.217603,94.181566,167.700557,203.495938,0.141895,...,72.320920,24.961298,41.585484,52.995487,52.067867,58.579530,32.826270,70.491200,98.744675,137.016560
70,female1_anxious_12b_2_cleaned.wav,female1_anxious_12b_2_cleaned.wav,208.698056,189.868424,58.983263,122.405354,167.210642,265.430348,321.169153,0.063167,...,174.814590,249.353230,191.225630,152.258900,114.009500,43.023643,102.886140,131.479720,120.498314,109.296040
153,female1_apologetic_3a_2_cleaned.wav,female1_apologetic_3a_2_cleaned.wav,171.245593,174.110113,12.888012,143.893358,162.216568,182.344498,193.187266,0.121150,...,27.859678,19.395012,32.565697,23.938278,57.986900,45.847023,31.045704,47.776592,31.087461,94.768820
1876,male2_anxious_14a_1_cleaned.wav,male2_anxious_14a_1_cleaned.wav,115.153673,112.896440,25.556122,51.763246,97.153194,137.395365,165.290064,0.180579,...,52.839190,51.807920,34.764988,15.470852,26.506105,18.104252,19.963308,15.986842,16.768785,26.095428
641,female2_angry_5a_2_cleaned.wav,female2_angry_5a_2_cleaned.wav,234.330910,231.070539,35.560203,125.992105,210.672207,268.514501,301.397336,0.179688,...,112.412030,187.183150,310.143950,513.543100,535.857850,411.974030,280.602700,440.596920,257.904600,135.725140
1213,male1_angry_13a_2_cleaned.wav,male1_angry_13a_2_cleaned.wav,168.594236,174.620243,29.005209,111.599714,147.469968,191.798022,215.596437,0.156714,...,44.555683,51.736534,77.826970,71.712814,120.969630,125.377910,64.402240,204.359740,222.124310,225.926320
1074,female2_happy_8b_1_cleaned.wav,female2_happy_8b_1_cleaned.wav,217.895644,203.495938,38.486619,178.179744,192.074589,237.841423,346.214624,0.172457,...,111.912360,34.764065,137.390730,123.172700,217.473880,204.357350,237.126200,126.809190,135.281460,157.096070
616,female2_angry_14a_1_cleaned.wav,female2_angry_14a_1_cleaned.wav,261.022451,257.874062,35.291358,184.463239,241.309964,289.473163,326.783091,0.217836,...,163.172470,474.000550,335.083620,486.886960,259.343500,239.872420,201.039260,165.232120,110.761604,61.116560


In [64]:
merged_df = merged_df.drop('fname', axis=1)
merged_df.head(5)

,filename,f0_mean,f0_median,f0_std,f0_0,f0_25,f0_75,f0_100,zcr_mean,zcr_var,...,mfcc21_var,mfcc22_var,mfcc23_var,mfcc24_var,mfcc25_var,mfcc26_var,mfcc27_var,mfcc28_var,mfcc29_var,mfcc30_var
0,female1_angry_10a_1_cleaned.wav,219.177008,216.220905,43.332156,129.683955,193.467046,245.171366,324.900959,0.103509,0.011014,...,131.51295,292.11954,364.214750,290.30600,293.173980,268.17877,383.54803,507.07217,298.34247,257.717350
1,female1_angry_10a_2_cleaned.wav,233.821368,223.845919,67.100423,128.194408,185.269329,271.634489,379.736848,0.117923,0.011924,...,263.28800,181.90977,184.546810,179.15126,130.375870,222.84843,209.25677,273.20135,121.19591,113.026375
2,female1_angry_10b_1_cleaned.wav,201.024166,185.531823,51.878638,129.683955,163.391545,213.739184,324.900959,0.109007,0.010337,...,110.76114,219.16455,139.534700,113.84319,52.278015,74.25708,135.96268,191.18411,140.03432,116.458626
3,female1_angry_11a_1_cleaned.wav,273.740375,270.069989,64.761666,156.916820,231.747560,318.410406,384.149178,0.170557,0.024981,...,295.06910,217.57916,242.317860,246.15382,159.066210,113.58967,138.29584,196.05043,108.37479,187.551960
4,female1_angry_10b_2_cleaned.wav,223.694014,218.101547,57.499029,125.266444,181.294271,265.430348,332.495158,0.116036,0.011352,...,283.61505,376.75586,126.861664,133.37468,215.111160,228.07380,208.39525,220.41928,158.63525,142.688310


In [65]:
merged_df.to_csv("/content/drive/MyDrive/LY Project/Dataframes/Features/JL-Corpus/concatenated_features_jlcorpus_with_filename.csv", index=False)

In [69]:
initial_path = list(df['filename'])
updated_cleaned_path = []

for entry in t: 
  entry += '_cleaned.wav'
  updated_cleaned_path.append(entry)

print(updated_cleaned_path)


['female1_angry_10a_1_cleaned.wav', 'female1_angry_10a_2_cleaned.wav', 'female1_angry_10b_1_cleaned.wav', 'female1_angry_10b_2_cleaned.wav', 'female1_angry_11a_1_cleaned.wav', 'female1_angry_11a_2_cleaned.wav', 'female1_angry_11b_1_cleaned.wav', 'female1_angry_11b_2_cleaned.wav', 'female1_angry_12a_1_cleaned.wav', 'female1_angry_12a_2_cleaned.wav', 'female1_angry_12b_1_cleaned.wav', 'female1_angry_12b_2_cleaned.wav', 'female1_angry_13a_1_cleaned.wav', 'female1_angry_13a_2_cleaned.wav', 'female1_angry_13b_1_cleaned.wav', 'female1_angry_13b_2_cleaned.wav', 'female1_angry_14a_1_cleaned.wav', 'female1_angry_14a_2_cleaned.wav', 'female1_angry_14b_1_cleaned.wav', 'female1_angry_14b_2_cleaned.wav', 'female1_angry_15a_1_cleaned.wav', 'female1_angry_15a_2_cleaned.wav', 'female1_angry_15b_1_cleaned.wav', 'female1_angry_15b_2_cleaned.wav', 'female1_angry_1a_1_cleaned.wav', 'female1_angry_1a_2_cleaned.wav', 'female1_angry_1b_1_cleaned.wav', 'female1_angry_1b_2_cleaned.wav', 'female1_angry_2a_1_cle

In [70]:
df.head()

,Unnamed: 0,path,filename,dataset,duration,sample_rate,gender,age,emotion
0,0,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_angry_10a_1,JL-Corpus,2.75,44100,female,30,angry
1,1,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_angry_10a_2,JL-Corpus,2.60,44100,female,30,angry
2,2,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_angry_10b_1,JL-Corpus,2.45,44100,female,30,angry
3,3,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_angry_10b_2,JL-Corpus,2.60,44100,female,30,angry
4,4,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_angry_11a_1,JL-Corpus,2.00,44100,female,30,angry


In [73]:
df['filename'] = updated_cleaned_path
df.head(5)

,Unnamed: 0,path,filename,dataset,duration,sample_rate,gender,age,emotion
0,0,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_angry_10a_1_cleaned.wav,JL-Corpus,2.75,44100,female,30,angry
1,1,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_angry_10a_2_cleaned.wav,JL-Corpus,2.60,44100,female,30,angry
2,2,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_angry_10b_1_cleaned.wav,JL-Corpus,2.45,44100,female,30,angry
3,3,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_angry_10b_2_cleaned.wav,JL-Corpus,2.60,44100,female,30,angry
4,4,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_angry_11a_1_cleaned.wav,JL-Corpus,2.00,44100,female,30,angry


In [74]:
merged_df.head(5)

,filename,f0_mean,f0_median,f0_std,f0_0,f0_25,f0_75,f0_100,zcr_mean,zcr_var,...,mfcc21_var,mfcc22_var,mfcc23_var,mfcc24_var,mfcc25_var,mfcc26_var,mfcc27_var,mfcc28_var,mfcc29_var,mfcc30_var
0,female1_angry_10a_1_cleaned.wav,219.177008,216.220905,43.332156,129.683955,193.467046,245.171366,324.900959,0.103509,0.011014,...,131.51295,292.11954,364.214750,290.30600,293.173980,268.17877,383.54803,507.07217,298.34247,257.717350
1,female1_angry_10a_2_cleaned.wav,233.821368,223.845919,67.100423,128.194408,185.269329,271.634489,379.736848,0.117923,0.011924,...,263.28800,181.90977,184.546810,179.15126,130.375870,222.84843,209.25677,273.20135,121.19591,113.026375
2,female1_angry_10b_1_cleaned.wav,201.024166,185.531823,51.878638,129.683955,163.391545,213.739184,324.900959,0.109007,0.010337,...,110.76114,219.16455,139.534700,113.84319,52.278015,74.25708,135.96268,191.18411,140.03432,116.458626
3,female1_angry_11a_1_cleaned.wav,273.740375,270.069989,64.761666,156.916820,231.747560,318.410406,384.149178,0.170557,0.024981,...,295.06910,217.57916,242.317860,246.15382,159.066210,113.58967,138.29584,196.05043,108.37479,187.551960
4,female1_angry_10b_2_cleaned.wav,223.694014,218.101547,57.499029,125.266444,181.294271,265.430348,332.495158,0.116036,0.011352,...,283.61505,376.75586,126.861664,133.37468,215.111160,228.07380,208.39525,220.41928,158.63525,142.688310


In [76]:
final_merged_df = pd.merge(df, merged_df, on='filename')
final_merged_df.head(5)

,Unnamed: 0,path,filename,dataset,duration,sample_rate,gender,age,emotion,f0_mean,...,mfcc21_var,mfcc22_var,mfcc23_var,mfcc24_var,mfcc25_var,mfcc26_var,mfcc27_var,mfcc28_var,mfcc29_var,mfcc30_var
0,0,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_angry_10a_1_cleaned.wav,JL-Corpus,2.75,44100,female,30,angry,219.177008,...,131.51295,292.11954,364.214750,290.30600,293.173980,268.17877,383.54803,507.07217,298.34247,257.717350
1,1,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_angry_10a_2_cleaned.wav,JL-Corpus,2.60,44100,female,30,angry,233.821368,...,263.28800,181.90977,184.546810,179.15126,130.375870,222.84843,209.25677,273.20135,121.19591,113.026375
2,2,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_angry_10b_1_cleaned.wav,JL-Corpus,2.45,44100,female,30,angry,201.024166,...,110.76114,219.16455,139.534700,113.84319,52.278015,74.25708,135.96268,191.18411,140.03432,116.458626
3,3,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_angry_10b_2_cleaned.wav,JL-Corpus,2.60,44100,female,30,angry,223.694014,...,283.61505,376.75586,126.861664,133.37468,215.111160,228.07380,208.39525,220.41928,158.63525,142.688310
4,4,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_angry_11a_1_cleaned.wav,JL-Corpus,2.00,44100,female,30,angry,273.740375,...,295.06910,217.57916,242.317860,246.15382,159.066210,113.58967,138.29584,196.05043,108.37479,187.551960


In [77]:
final_merged_df.sample(5)

,Unnamed: 0,path,filename,dataset,duration,sample_rate,gender,age,emotion,f0_mean,...,mfcc21_var,mfcc22_var,mfcc23_var,mfcc24_var,mfcc25_var,mfcc26_var,mfcc27_var,mfcc28_var,mfcc29_var,mfcc30_var
392,392,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,female1_excited_3a_1_cleaned.wav,JL-Corpus,2.30,44100,female,30,excited,269.833928,...,432.682160,367.828200,436.595950,311.673520,257.780730,342.514430,148.208950,195.802120,116.921130,116.635120
1431,1431,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,male1_assertive_7b_2_cleaned.wav,JL-Corpus,2.00,44100,male,30,assertive,115.697625,...,42.440994,55.738834,41.213320,28.264536,38.951073,46.240690,26.436829,36.336987,40.526250,28.174057
2296,2296,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,male2_neutral_14a_1_cleaned.wav,JL-Corpus,1.75,44100,male,30,neutral,93.685557,...,28.002672,25.523500,17.711006,31.925728,22.803923,48.642593,18.537971,14.668794,17.221458,20.380783
1253,1253,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,male1_angry_8a_2_cleaned.wav,JL-Corpus,2.15,44100,male,30,angry,147.238645,...,27.794620,49.286224,27.373604,61.269270,51.844730,37.319730,20.298820,61.287120,77.799430,69.552830
2012,2012,/kaggle/input/jl-corpus/Raw JL corpus (uncheck...,male2_assertive_3a_1_cleaned.wav,JL-Corpus,2.00,44100,male,30,assertive,104.397693,...,41.455605,42.863373,29.111914,21.125187,16.665585,25.523570,25.412240,21.334423,11.795935,25.129120


In [79]:
final_merged_df.to_csv("/content/drive/MyDrive/LY Project/Dataframes/Features/JL-Corpus/FINAL_DF_JLCORPUS.csv", index=False)

<br>[Back to top](#top2)